In [3]:
from Bio import SeqIO
for seq_record in SeqIO.parse("FANCF.fasta", "fasta"):
    my_seq=repr(seq_record.seq)
    tamanho =len(seq_record)
    print(seq_record.id)
    print(my_seq)
    print(tamanho)

NC_000011.10:c22625823-22622533
Seq('CTTCGCGCACCTCATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAGCT...GGA', SingleLetterAlphabet())
3291


In [ ]:
"com a sequencia em modo fasta, vamos fazer o Blast"

In [5]:
from Bio.Blast import NCBIWWW
fasta_string = open("FANCF.fasta").read()
result_handle = NCBIWWW.qblast("blastn", "nt", fasta_string)
with open("my_blast.xml", "w") as out_handle:
    out_handle.write(result_handle.read())
result_handle.close()
result_handle = open ("my_blast.xml")

In [ ]:
" vamos fazer parse ao ficheiro xml e imprimir os alinhamento com e value menor que 0.04"

In [9]:
from Bio.Blast import NCBIXML
result_handle = open ("my_blast.xml")
blast_records = NCBIXML.parse(result_handle)
e_value = 0.04
#L= []

for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < e_value:
                print("alinhamento")
                print("sequencia:", alignment.title)
                print("tamanho:", alignment.length)
                print ("evalue:", hsp.expect)
                print(hsp.query[0:100]+ "...")
                print (hsp.match[0:100] + "...")
                print (hsp.sbjct[0:100] + "...", "\n")
            #L.append(alignment.title)
#print (L)            


#for blast_record in blast_records:
    #print (blast_records)

alinhamento
sequencia: gi|1732746308|ref|NM_022725.4| Homo sapiens FA complementation group F (FANCF), mRNA
tamanho: 3291
evalue: 0.0
CTTCGCGCACCTCATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAGCTTCTGGCGGTCTCAAGCACTACCTACGTCAGCACCTGGGACCCCGCC...
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
CTTCGCGCACCTCATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAGCTTCTGGCGGTCTCAAGCACTACCTACGTCAGCACCTGGGACCCCGCC... 

alinhamento
sequencia: gi|168480095|ref|NG_007425.1| Homo sapiens FA complementation group F (FANCF), RefSeqGene (LRG_527) on chromosome 11
tamanho: 10309
evalue: 0.0
CTTCGCGCACCTCATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAGCTTCTGGCGGTCTCAAGCACTACCTACGTCAGCACCTGGGACCCCGCC...
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
CTTCGCGCACCTCATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAGCTTCTGGCGGTCTCAAGCACTACCTACGTCAGCACCTGGGACCCCGCC... 

alinhamento
sequencia: gi|168480095|ref|NG_007425.1| Homo sapiens FA co

sequencia: gi|1327781222|ref|XM_003804052.3| PREDICTED: Otolemur garnettii Fanconi anemia complementation group F (FANCF), mRNA
tamanho: 3523
evalue: 4.60721e-149
AAGTAGAAGATCCAGGTTTTAGTGCAGCTTCTGCCGTTAACTAGGTTAATAAATCACAAC---CTTGGGGCCACAGTTGCCTTATATGTAAATGAAGTGT...
|| |||||||||||||||||||||   |||||| | ||||||||||||||||||||| |    ||| | |  ||| ||||||||| |       ||||||...
AATTAGAAGATCCAGGTTTTAGTGTTCCTTCTGTCCTTAACTAGGTTAATAAATCACCAGTCACTTAGAGTTACATTTGCCTTATGTT------AAGTGT... 

alinhamento
sequencia: gi|829901663|ref|XM_012735530.1| PREDICTED: Microcebus murinus Fanconi anemia complementation group F (FANCF), mRNA
tamanho: 1246
evalue: 0.0
CTTCGCGCACCTCATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAGCTTCTGGCGGTCTCAAGCACTACCTACGTCAGCACCTGGGACCCCGCC...
||| |||  |||||||||| | || || |||||||| || |||||||| ||| |||| | |||||| ||||||||| ||||||||||||||||||| |||...
CTTTGCGTGCCTCATGGAACCGCTCCTACAGCACCTCGAGCGCTTTTCAGAGGTTCTAGTGGTCTCGAGCACTACCCACGTCAGCACCTGGGACCCTGCC... 

alinhamento
sequencia: gi|946690236|ref|XM_0

In [ ]:
"com blast feito, vamos procurar informação, como o número de hits (49, neste caso), e para cada hit
"conseguimos ver quantos High Scoring Pairs tem, o ID e um pouco da discrição"

In [38]:
from Bio import SearchIO

blast_qresult = SearchIO.read("my_blast.xml", "blast-xml")
print (blast_qresult)

Program: blastn (2.10.0+)
  Query: NC_000011.10:c22625823-22622533 (3291)
         Homo sapiens chromosome 11, GRCh38.p13 Primary Assembly
 Target: nt_v5
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  gi|1732746308|ref|NM_022725.4|  Homo sapiens FA complem...
            1      3  gi|168480095|ref|NG_007425.1|  Homo sapiens FA compleme...
            2     37  gi|22038636|gb|AC103801.2|  Homo sapiens chromosome 11,...
            3     30  gi|28195912|gb|AC116010.5|  Homo sapiens chromosome 11,...
            4      2  gi|59860522|gb|AY928335.1|  Homo sapiens Fanconi anemia...
            5      1  gi|10434946|dbj|AK023153.1|  Homo sapiens cDNA FLJ13091...
            6      1  gi|77799890|dbj|AK001716.2|  Homo sapiens cDNA FLJ10854...
            7      2  gi|1220173480|ref|XM_003910119.4|  PREDICTED: Papio anu.

In [ ]:
" selecionando as linhas com mais HSP, vamos alinhar o hit com a sequencia"

In [46]:
blast_hsp37 = blast_qresult[2][0]
blast_hsp30 = blast_qresult[3][0]
print(blast_hsp37, "\n", "\n", blast_hsp30)

      Query: NC_000011.10:c22625823-22622533 Homo sapiens chromosome 11, GRCh...
        Hit: gi|22038636|gb|AC103801.2| Homo sapiens chromosome 11, clone RP1...
Query range: [0:3291] (1)
  Hit range: [163266:166557] (1)
Quick stats: evalue 0; bitscore 5936.17
  Fragments: 1 (3291 columns)
     Query - CTTCGCGCACCTCATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAGCTTCTGG~~~GAGGA
             |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||~~~|||||
       Hit - CTTCGCGCACCTCATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAGCTTCTGG~~~GAGGA 
 
       Query: NC_000011.10:c22625823-22622533 Homo sapiens chromosome 11, GRCh...
        Hit: gi|28195912|gb|AC116010.5| Homo sapiens chromosome 11, clone RP1...
Query range: [0:3291] (1)
  Hit range: [105646:108937] (1)
Quick stats: evalue 0; bitscore 5936.17
  Fragments: 1 (3291 columns)
     Query - CTTCGCGCACCTCATGGAATCCCTTCTGCAGCACCTGGATCGCTTTTCCGAGCTTCTGG~~~GAGGA
             |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||~~~|||||
       Hi